In [ ]:
# Instalar librerías necesarias
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os

In [ ]:
print("GPU Disponible:", tf.config.list_physical_devices('GPU'))

GPU Disponible: []


In [ ]:
# ========== Configuración general ==========
SEQ_LENGTH = 30
VALIDATION_SPLIT = 0.1
EPOCHS = 30
BATCH_SIZE = 32
MODEL_DIR = "modelos_lstm_tuned"
TUNER_DIR = "keras_tuner_dir"
os.makedirs(MODEL_DIR, exist_ok=True)

# ========== Cargar y escalar datos ==========
df = pd.read_csv("SP500_HMM_States.csv")
df['Date'] = pd.to_datetime(df['Date'])

if 'Return' not in df.columns or 'State' not in df.columns:
    raise ValueError("El archivo debe contener 'Return' y 'State'.")

scaler = StandardScaler()
df['Return_scaled'] = scaler.fit_transform(df[['Return']])

# ========== Crear generador ==========
def create_generator(data_array):
    split_idx = int(len(data_array) * (1 - VALIDATION_SPLIT))
    train_data, val_data = data_array[:split_idx], data_array[split_idx:]
    train_gen = TimeseriesGenerator(train_data, train_data, length=SEQ_LENGTH, batch_size=BATCH_SIZE)
    val_gen = TimeseriesGenerator(val_data, val_data, length=SEQ_LENGTH, batch_size=BATCH_SIZE)
    return train_gen, val_gen

# ========== Definir espacio de hiperparámetros ==========
def build_model(hp):
    model = Sequential()

    # LSTM 1
    model.add(LSTM(
        units=hp.Int('lstm_units_1', 32, 128, step=32),
        return_sequences=True,
        activation='tanh',
        input_shape=(SEQ_LENGTH, 1)
    ))

    if hp.Boolean('use_batchnorm'):
        model.add(BatchNormalization())

    model.add(Dropout(hp.Float('dropout_1', 0.1, 0.5, step=0.1)))

    # LSTM 2
    model.add(LSTM(
        units=hp.Int('lstm_units_2', 16, 64, step=16),
        activation='tanh'
    ))

    model.add(Dropout(hp.Float('dropout_2', 0.1, 0.5, step=0.1)))
    model.add(Dense(1))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        ),
        loss='mse'
    )

    return model

# ========== Función para optimizar y entrenar ==========
def optimize_and_train(name, data_array):
    print(f"\n🔍 Optimización de hiperparámetros para: {name}")

    train_gen, val_gen = create_generator(data_array)

    tuner = kt.Hyperband(
        build_model,
        objective='val_loss',
        max_epochs=EPOCHS,
        factor=3,
        directory=TUNER_DIR,
        project_name=f'tuning_{name}'
    )

    tuner.search(train_gen, validation_data=val_gen, epochs=EPOCHS,
                 callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
                 verbose=1)

    best_hp = tuner.get_best_hyperparameters(1)[0]

    print(f"\n✅ Mejores hiperparámetros para {name}:")
    for k, v in best_hp.values.items():
        print(f"  - {k}: {v}")

    # Entrenar modelo final
    model = tuner.hypermodel.build(best_hp)
    model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS,
              callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
              verbose=1)

    model.save(f"{MODEL_DIR}/LSTM_{name}.h5")
    print(f"💾 Modelo guardado: {MODEL_DIR}/LSTM_{name}.h5")



In [ ]:
# ========== Entrenar modelo general ==========
full_data = df['Return_scaled'].values.reshape(-1, 1)
optimize_and_train("Full_Series", full_data)

# ========== Entrenar modelos por estado ==========
for state in sorted(df['State'].unique()):
    state_data = df[df['State'] == state]['Return_scaled'].values
    if len(state_data) <= SEQ_LENGTH + 10:
        print(f"⚠️ Estado {state} omitido (muy pocos datos).")
        continue

    try:
        state_array = state_data.reshape(-1, 1)
        optimize_and_train(f"State_{state}", state_array)
    except Exception as e:
        print(f"❌ Error en State {state}: {e}")


Trial 90 Complete [00h 01m 51s]
val_loss: 0.9437922835350037

Best val_loss So Far: 0.9328762292861938
Total elapsed time: 00h 44m 40s

✅ Mejores hiperparámetros para State_3:
  - lstm_units_1: 128
  - use_batchnorm: True
  - dropout_1: 0.30000000000000004
  - lstm_units_2: 48
  - dropout_2: 0.1
  - learning_rate: 0.0001740001673618082
  - tuner/epochs: 30
  - tuner/initial_epoch: 10
  - tuner/bracket: 1
  - tuner/round: 1
  - tuner/trial_id: 0075
Epoch 1/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - loss: 1.3800 - val_loss: 0.9837
Epoch 2/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - loss: 1.3714 - val_loss: 0.9824
Epoch 3/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - loss: 1.3031 - val_loss: 0.9825
Epoch 4/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 1.2672 - val_loss: 0.9870
Epoch 5/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - loss: 1.3024 - val_loss: 0.9795
Epoch 6/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 1.3769 - val_loss: 0.9928
Epoch 7/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 

💾 Modelo guardado: modelos_lstm_tuned/LSTM_State_3.h5

🔍 Optimización de hiperparámetros para: State_4
❌ Error en State 4: `start_index+length=30 > end_index=19` is disallowed, as no part of the sequence would be left to be used as current step.


In [ ]:
import shutil
shutil.make_archive('/content/keras_tuner_dir', 'zip', '/content/keras_tuner_dir')
shutil.make_archive('/content/modelos_lstm_tuned', 'zip', '/content/modelos_lstm_tuned')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>